<a href="https://colab.research.google.com/github/yfhkucing/ComputerVision/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [8]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [10]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), strides=1, padding='same', 
 activation='relu', input_shape=(28,28,1))) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten()) 
model.add(Dense(64, activation='relu')) 
model.add(Dense(10, activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 64)                2

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [13]:
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


469/469 [==============================] - 16s 8ms/step - loss: 0.2309 - sparse_categorical_accuracy: 0.9316 - val_loss: 0.0666 - val_sparse_categorical_accuracy: 0.9791
Epoch 2/6
469/469 [==============================] - 3s 5ms/step - loss: 0.0606 - sparse_categorical_accuracy: 0.9813 - val_loss: 0.0482 - val_sparse_categorical_accuracy: 0.9840
Epoch 3/6
469/469 [==============================] - 3s 6ms/step - loss: 0.0426 - sparse_categorical_accuracy: 0.9869 - val_loss: 0.0382 - val_sparse_categorical_accuracy: 0.9873
Epoch 4/6
469/469 [==============================] - 3s 5ms/step - loss: 0.0318 - sparse_categorical_accuracy: 0.9898 - val_loss: 0.0375 - val_sparse_categorical_accuracy: 0.9874
Epoch 5/6
469/469 [==============================] - 3s 7ms/step - loss: 0.0260 - sparse_categorical_accuracy: 0.9917 - val_loss: 0.0328 - val_sparse_categorical_accuracy: 0.9884
Epoch 6/6
469/469 [==============================] - 3s 6ms/step - loss: 0.0223 - sparse_categorical_accuracy: 0.9

In [17]:
model.save('model.h5')